# Task 1 : Model Development and Experimentation



## 1. Approach and Model Selection

### 1.1 Understanding the constraints

With the constraint of running the models in an edge device (Raspberry Pi 5) within a 5-10ms target, we are limiting our experiments to the smallest parameter models of each major YOLO version, specifically the "nano" variants. A 5-10ms (100-200 FPS) target is extremely aggressive and will likely require post-training quantization (e.g., to INT8) and a highly optimized inference runtime (like ONNX or TFLite).

The goal is to find the model that offers the absolute best trade-off between accuracy (mAP) and computational cost (GFLOPs/Parameters). A model with low computational cost is the most likely to meet our speed target after quantization.

### 1.2 Model Candidate Justifications

- YOLOv6-Nano : Designed specifically for "hardware-friendly" deployment and industrial use cases, claiming state-of-the-art speed.
- YOLOv8-Nano : Successor to the v5, introduced an anchor-free design and a decoupled head, which provided a significant boost in the accuracy-speed trade-off.
- YOLOv10-Nano : Eliminated the NMS post-processing, which offers a strong theoretical contender for our low-latency 5-10ms target
- YOLOv11-Nano : Latest stable model from Ultralytics and successor to v8, offering higher mAP for the same or lower latency than all previous versions

There are also other YOLO versions (such as v9 or v12) not included for this experiment due to their design purpose of having better performance for the cost of latency, which does not fit our constraints. For a quick illustration of the speed and performance trade of each model versions you may refer to the [Ultralytic website](https://docs.ultralytics.com/models/yolo11/#supported-tasks-and-modes)

## 2. Model Development, Evaluation and Analysis

### 2.1 Training

In [ ]:
# !pip install -U ultralytics wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 54.7 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.22.3
    Uninstalling wandb-0.22.3:
      Successfully uninstalled wandb-0.22.3


In [ ]:
# !pip install --upgrade sympy==1.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.8.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.


In [ ]:
# !git clone https://github.com/RandyTunru/bottle_caps_detection.git

Cloning into 'bottle_caps_detection'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 0), reused 40 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 1.29 MiB | 4.20 MiB/s, done.


In [ ]:
# !yolo settings wandb=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Updated 'wandb=True'
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [5]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: randytunru (randytunru-hasanuddin-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
from google.colab import userdata
import os

os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

DATASET_YAML = os.path.abspath('/content/bottle_caps_detection/bottle_cap_dataset/dataset.yaml')

EPOCHS = 100
BATCH_SIZE = 4
WANDB_PROJECT = "Bottle-Cap-Detection"

In [7]:
from ultralytics import YOLO

In [8]:
model_v8n = YOLO('yolov8n.pt')

model_v8n.train(
    data=DATASET_YAML,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=640,
    project=WANDB_PROJECT,
    name='YOLOv8-Nano',
    exist_ok=True,
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/bottle_caps_detection/bottle_cap_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOv8-Nano, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

lr/pg0,▁▂▂▃▃▄▅▅▆▆▇▇███▇▇▇▇▇▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁
lr/pg1,▁▂▃▃▃▅▆▇▇▇█████▇▇▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁
lr/pg2,▁▂▃▄▄▅▅▅▆▆▇████▇▇▇▇▇▇▆▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁
metrics/mAP50(B),▁▁▁▁▁▁▁▃▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███████████████
metrics/mAP50-95(B),▁▁▁▁▁▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███████████████
metrics/precision(B),▁▁▁▁▁▁▁▅▄▆▃▅▅▅▅▅▅▅▅▅█████████████▇▇█████
metrics/recall(B),▁▁▁▁▁▄▆▆▂▂███▅▅▆▆▆███▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c13652cce00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [9]:
model_v10n = YOLO('yolov10n.pt')

model_v10n.train(
    data=DATASET_YAML,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=640,
    project=WANDB_PROJECT,
    name='YOLOv10-Nano',
    exist_ok=True,
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/bottle_caps_detection/bottle_cap_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOv10-Nano, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics

lr/pg0,▁▂▂▃▄▅▆▆▇▇▇▇███████▇▇▆▆▆▆▅▅▅▄▄▃▃▃▂▂▂▂▁▁▁
lr/pg1,▂▂▄▄▄▅▅▇▇▇█████▇▇▇▇▇▆▆▆▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁
lr/pg2,▂▂▃▃▃▄▄▅▅▆▆▇▇███▇▇▇▇▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▁▁
metrics/mAP50(B),▁▁▁▁▁▁▁▂▂▄▅▅▅▅▅▅▆▆▅▅▅▅▅▅▅▅▅▅▅▅▇▇▇▇██████
metrics/mAP50-95(B),▁▁▁▁▁▁▂▂▄▄▄▄▄▅▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▇█████▇▇
metrics/precision(B),▁▁▁▁▁▁▁▁▂▁▁▁▁▇▇▇██▃▃▃▃▃▆▆▆██████████▄▄▄▅
metrics/recall(B),▁▁▁▃▅▆▆▇▇▇▃▃▃▄▄▅▅▄▄▄▅▅▅▄▄▄▄▄▄▅▅▅▅██████▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c130a2b2ff0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [10]:
model_v11n = YOLO('yolo11n.pt')

model_v11n.train(
    data=DATASET_YAML,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=640,
    project=WANDB_PROJECT,
    name='YOLOv11-Nano',
    exist_ok=True,
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/bottle_caps_detection/bottle_cap_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOv11-Nano, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

wandb: WARNING Tried to log to step 100 that is less than the current step 101. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


lr/pg0,▁▁▂▃▃▄▄▅▇▇█████▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁
lr/pg1,▂▂▃▄▄▅▅▆▆▇▇▇█████▇▇▇▆▆▆▆▅▄▄▄▄▃▃▂▂▂▂▂▁▁▁▁
lr/pg2,▁▁▃▃▃▅▆▆▇▇▇███████▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▁
metrics/mAP50(B),▁▂▂▃▃▃▃▃▃▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████
metrics/mAP50-95(B),▁▁▁▂▃▂▂▂▂▃▅▅▇▇▇▇▇▇▇▆▆▆▆▆▇▇▇▇▇███████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇██▇▇▇▇▇▇▇▇▇▇▇
metrics/recall(B),▁▁████████▃▃▃▄▆▆▇▇▇▇███████▇▇▇██████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c1303489490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [12]:
model_v6n = YOLO('yolov6n.yaml')

model_v6n.train(
    data=DATASET_YAML,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=640,
    project=WANDB_PROJECT,
    name='YOLOv6-Nano',
    exist_ok=True,
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/bottle_caps_detection/bottle_cap_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov6n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOv6-Nano, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

Overriding model.yaml nc=80 with nc=3
activation: torch.nn.ReLU()

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  2     18560  ultralytics.nn.modules.conv.Conv             [32, 32, 3, 1]                
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  4    147968  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 1]                
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  6    886272  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 1]              
  7                  

lr/pg0,▁▂▂▃▃▅▅▆▇▇██████▇▇▇▇▆▆▆▆▅▅▅▄▄▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▁▂▃▃▃▅▅▅▆▇▇▇▇██▇▇▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▂▂▁▁▁▁
lr/pg2,▁▁▂▃▃▄▄▅▅▅▆▇▇▇█████▇▇▇▇▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▃▅▅▅▆▆▆▆▆▆▆▆▆▆█████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▄▄▄▄▄▄▅▅▄▄▄▄▄▄▄
metrics/recall(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▂▂▂▂▅▅▅▇▇▇▇█████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c1302f40980>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

### 2.2 Model Evaluation and Analysis

All four "nano" models were trained on our 3-class dataset for 100 epochs without modifying the models original training recipe, with all results logged to Weights & Biases. Below are a summary of the experiment results

Localization (mAP50-95):

- YOLOv8n (purple) was the winner providing the best result for this dataset.

- YOLOv11n (orange) was a close second.

- YOLOv10n (orange-red) and YOLOv6n (pink) performed significantly worse, with v6n showing the lowest accuracy.

Class accuracy (Precision and Recall):

- Both YOLOv8n and YOLOv11n performed the best, with v8 offering better precision (less false positives) while v11 offers better recall (less false negatives)

Efficiency (GFLOPs & Parameters):

- YOLOv11n was the most efficient model, having the lowest GFLOPs (\~6.5) and fewest parameters (\~2.5M).

- YOLOv6n was the least efficient, with the highest GFLOPs and most parameters.

Below is a table of the pros and cons of each model

| Models    | Pros                                                                                            | Cons                                                     |   |   |
|-----------|-------------------------------------------------------------------------------------------------|----------------------------------------------------------|:-:|---|
| YOLOv8-n  | - Highest precision (less false positives)<br>- Best localization (higher mAP50-95)             | - Less Efficient (High parameter count and GFLOPS)       |   |   |
| YOLOv11-n | - Highest recall (less false negatives)<br>- Most Efficient (Lowest parameter count and GFLOPS) | - Slightly less optimal localization                     |   |   |
| YOLOv10-n | None in this experiment                                                                         | - Similar efficiency with v8 with much worse performance |   |   |
| YOLOv6-n | None in this experiment                                                                         | - Significantly worse efficiency and performance         |   |   |

With this information, the model we chose is the YOLOv11n, as it achieved optimal performance while using less computation than the other models. Therefore, for the subsequent inference and ONNX benchmarking, we will only proceed with this model.

## 3. Additional insights

With only 12 images, this project suffers from lack of data and the models most likely overfitted. Not to mention the images provided doesn't cover different cases such as frames where there are multiple different classes present in the frame.